In [1]:
!pip install gradio
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.7/65.7 kB 6.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.0/289.0 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 3.8 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of mdit-py-plugins to determine which version is compatible with other requirements. This could take a while.
  

In [2]:
#이미지에 클래스가 안뜨는 웹
import gradio as gr
from ultralytics import YOLO
from google.colab.patches import cv2_imshow
import cv2
import os

model=YOLO('/content/drive/MyDrive/코랩/dataflow/best.pt')

def imgflow(img):
  img=img.name
  res=model.predict(img, conf=0.5)
  wh=res[0].boxes.xywh
  xy=res[0].boxes.xyxy

  tup=[]

  img1=cv2.imread(img,cv2.IMREAD_COLOR )

  for i in range(len(wh)):
    tup1=[]
    wh1=wh[i].tolist()
    xy1=xy[i].tolist()
    tup1.append(round(xy1[0]))
    tup1.append(round(xy1[1]))
    tup1.append(round(wh1[2]))
    tup1.append(round(wh1[3]))
    tup.append(tup1)
  for i in range(len(tup)):
    cv2.rectangle(img1,tuple(tup[i]),(0,255,0),2)
  return img1

def clsif(img):
  img=img.name
  classes={0.0:'G_dot',1.0:'c_circle',2.0:'c_curve',3.0:'c_cut',4.0:'c_diagonal',5.0:'c_empty', 6.0:'c_inner',7.0:'c_jut',8.0:'c_thick',9.0:'c_thin',10.0:'curve',11.0:'dot',12.0:'scratch'}
  res=model.predict(img, conf=0.5)
  cls=res[0].boxes.cls.tolist()
  result=[]
  for i in range(len(cls)):
    result.append(classes.get(cls[i]))
  return result

def imgshow(img):
  return img.name

with gr.Blocks() as demo:
    with gr.Row():
      im = gr.File(label='입력파일')
      out = gr.Image(label='원본이미지')
      im.change(imgshow, im, out)
    with gr.Row():
      btn = gr.Button(value="예측하기", size=10)
      btn1 = gr.Button(value="설명보기", size=10)
    with gr.Row():
      im_2 = gr.Image(label='예측이미지')
      txt=gr.TextArea(label='결함 정보')
    btn.click(imgflow, inputs=[im], outputs=[im_2])
    btn1.click(clsif, inputs=[im], outputs=[txt])

demo.launch(share=True)

FileNotFoundError: ignored

In [ ]:
#이미지에 클래스와 성능이 나오는 웹
import gradio as gr
from ultralytics import YOLO
from google.colab.patches import cv2_imshow
import cv2
import os

#학습한 욜로 모델의 경로 입력
model=YOLO('/content/drive/MyDrive/코랩/dataflow/

           t')

#바운딩 박스 이미지 보여주는 함수
def imgflow(img):
  img=img.name
  res=model.predict(img, conf=0.5)
  imsh=res[0].plot()
  return imsh

#결함 정보를 보여주는 함수
def clsif(img):
  img=img.name
  classes={0.0:'G_dot',1.0:'c_circle',2.0:'c_curve',3.0:'c_cut',4.0:'c_diagonal',5.0:'c_empty', 6.0:'c_inner',7.0:'c_jut',8.0:'c_thick',9.0:'c_thin',10.0:'curve',11.0:'dot',12.0:'scratch'}
  res=model.predict(img, conf=0.5)
  cls=res[0].boxes.cls.tolist()
  result=[]
  # for i in range(len(cls)):
  #   result.append(classes.get(cls[i]))
  value_counts = {}  # 빈 딕셔너리를 생성하여 각 값의 개수를 저장합니다.
  for value in cls:
      if value in value_counts:
          value_counts[value] += 1
      else:
          value_counts[value] = 1

  for j in range(len(classes)):
    for i in range(len(value_counts)):
      if list( value_counts.keys())[i]==list(classes.keys())[j]:
        value_counts[classes.get(list(classes.keys())[j])] = value_counts.pop(list(value_counts.keys())[i])
  result_str = ""
  for key, value in value_counts.items():
      result_str += f"{key}는 {value}개\n"
  return result_str

#원본 이미지 보여주는 함수
def imgshow(img):
  return img.name

with gr.Blocks() as demo:
    with gr.Row():
      im = gr.File(label='입력파일')
      out = gr.Image(label='원본이미지')
      im.change(imgshow, im, out)
    with gr.Row():
      btn = gr.Button(value="예측하기", size=10)
      btn1 = gr.Button(value="설명보기", size=10)
    with gr.Row():
      im_2 = gr.Image(label='예측 이미지')
      txt=gr.TextArea(label='결함 정보')
    btn.click(imgflow, inputs=[im], outputs=[im_2])
    btn1.click(clsif, inputs=[im], outputs=[txt])

demo.launch(share=True)

In [ ]:
val={1:100, 2:200, 3:300}
val2={1:'a', 2:'b', 3:'c'}

if list(val.keys())[0]==list(val2.keys())[0]:
  val[val2.get(list(val2.keys())[1])] = val.pop(list(val.keys())[1])
val